## Preamble

Initialize some functions for better control of the code

In [2]:
import time
from memory_profiler import memory_usage

In [3]:
def timing(func, *args, **kwargs):
    start = time.time_ns()
    result = func(*args, **kwargs)
    end = time.time_ns()
    elapsed_ns = end - start
    timestamp = str(elapsed_ns)
    timestamp = timestamp.zfill((len(timestamp) // 3 + 1) * 3)
    timestamp = ' '.join(timestamp[i:i+3] for i in range(0, len(timestamp), 3))
    print(f'Time spent in {func.__name__} is {timestamp} ns')
    return result

def timing_wrap(func):
    def wrapper(*args, **kwargs):
        return timing(func, *args, **kwargs)
    return wrapper

In [4]:
def max_memory(func, *args, **kwargs):
    mem_used, result = memory_usage((func, args, kwargs), retval=True)
    print(f'Maximum memory usage in {func.__name__} is {max(mem_used)} MiB')
    return result

def max_memory_wrap(func):
    def wrapper(*args, **kwargs):
        return max_memory(func, *args, **kwargs)
    return wrapper

# 2. Introduction: theory and practice

### 2.2 Fibonacci numbers

#### Task 1

Дано целое число $1 \leq n \leq 40$, необходимо вычислить $n$-е число Фибоначчи (напомним, что $F_0=0, F_1=1, F_n=F_{n-1}+F_{n-2}$ при $n\geq2$).

**Sample Input**:
3

**Sample Output**:
2

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [5]:
def fib(n):
    fib_values = [1, 1]
    for i_n in range(2, n):
        fib_values.append(fib_values[i_n-1] + fib_values[i_n-2])
    return fib_values[n - 1]

def main():
    n = int(input())
    print(fib(n))

##### Checking

In [6]:
fib(1), fib(2), fib(3), fib(20), fib(40)

(1, 1, 2, 6765, 102334155)

#### Task 2

Дано число $1 \leq n \leq 10^7$, необходимо найти последнюю цифру $n$-го числа Фибоначчи.

Как мы помним, числа Фибоначчи растут очень быстро, поэтому при их вычислении нужно быть аккуратным с переполнением. В данной задаче, впрочем, этой проблемы можно избежать, поскольку нас интересует только последняя цифра числа Фибоначчи: если $1 \leq a, b \leq 9$ — последние цифры чисел $F_i$ и $F_{i+1}$ соответственно, то $(a+b)\ mod\ 10$ — последняя цифра числа $F_{i+2}$

**Sample Input**:
841645

**Sample Output**:
5

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [7]:
def fib_digit(n):
    n_2 = 1
    n_1 = 1
    n_0 = (n_2 + n_1) % 10
    for i_n in range(2, n):
        n_0 = (n_2 + n_1) % 10
        n_2 = n_1
        n_1 = n_0
    return n_0


def main():
    n = int(input())
    print(fib_digit(n))

##### Checking

In [8]:
fib_digit(841645)

5

#### Task 3

Даны целые числа $1 \leq n \leq 10^{18}$ и $2 \leq m \leq 10^5$, необходимо найти остаток от деления $n$-го числа Фибоначчи на $m$.

**Sample Input**:
10 2

**Sample Output**:
1

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [9]:
def fib_mod(n, m):
    v1, v2, v3 = 1, 1, 0

    for rec in bin(n)[3:]: # Matrix Exponentiation
        calc = (v2*v2) % m
        v1, v2, v3 = (v1*v1+calc) % m, ((v1+v3)*v2) % m, (calc+v3*v3) % m
        if rec == '1':
            v1, v2, v3 = (v1+v2) % m, v1, v2

    return v2

    # previous solution
    # fib_mods = [0, 1]
    # prev, curr = 0, 1
    #
    # for i_n in range(6 * m):
    #     prev, curr = curr, (prev + curr) % m
    #     fib_mods.append(curr % m)
    #     if fib_mods[-1] == 1 and fib_mods[-2] == 0:
    #         break
    # return fib_mods[n % (len(fib_mods)-2)]


def main():
    n, m = map(int, input().split())
    print(fib_mod(n, m))

##### Checking

In [10]:
timing(fib_mod, 10, 2), timing(fib_mod, 10**18, 10**5)

Time spent in fib_mod is 000 ns
Time spent in fib_mod is 000 ns


(1, 46875)

### 2.3 The greatest common divisor

#### Task 1

По данным двум числам $1 \leq a, b \leq 2 * 10^9$ найдите их наибольший общий делитель.

**Sample Input 1**:
18 35

**Sample Output 1**:
1

**Sample Input 2**:
14159572 63967072

**Sample Output 2**:
4

**Time Limit**: 3 секунды
**Memory Limit**: 256 MB

##### Solution

In [11]:
def gcd(a, b):
    if a == 0:
        return b
    if b == 0:
        return a
    if a >= b:
        return gcd(a % b, b)
    if b >= a:
        return gcd(a, b % a)


def main():
    a, b = map(int, input().split())
    print(gcd(a, b))

##### Checking

In [14]:
gcd(18, 35), timing(gcd, 14159572, 63967072)

Time spent in gcd is 000 ns


(1, 4)